*UE Learning from User-generated Data, CP MMS, JKU Linz 2022*
# Exercise 4: Evaluation

In this exercise we'll have a closer look at two very different RecSys evaluation metrics and use them to compare the three algorithms we implemented so far to each other. Please consult the lecture slides and the presentation from UE Session 4 for a recap.

The assignment submission deadline is 15.05.2022 23:59.

Make sure to rename the notebook according to the convention:

LUD22_ex03_k<font color='red'><Matr. Number\></font>_<font color='red'><Surname-Name\></font>.ipynb

for example:

LUD22_ex03_k000007_Bond_James.ipynb

## Implementation
In this exercise, as before, you are reqired to write a number of functions. Only implemented functions are graded. Insert your implementations into the templates provided. Please don't change the templates even if they are not pretty. Don't forget to test your implementation for correctness and efficiency.

Please **only use libraries already imported in the notebook**.

In [1]:
import pandas as pd
import numpy as np
import random as rnd

import torch
from torch import nn, optim
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import scipy.linalg as linalg

from tqdm import tqdm
import random as rnd

## <font color='red'>TASK 1/2</font>: Evaluation Metrics

Implement DCG, nDCG and Average Artist entropy in the corresponding templates.

### DCG Score
Implement DCG following the input/output convention:
#### Input:
* prediction - (not an interaction matrix!) numpy array with recommendations. Row index corresponds to User_id, column index corresponds to the rank of the item mentioned in the sell. Every cell (i,j) contains **item id** recommended to the user (i) on the position (j) in the list. For example:

The following predictions structure [[12, 7, 99], [0, 97, 6]] means that the user with id==1 (second row) got recommended item **0** on the top of the list, item **97** on the second place and item **6** on the third place.

* test_interaction_matrix - (plain interaction matrix format as before!) interaction matrix constructed from interactions held out as a test set, rows - users, columns - items, cells - 0 or 1

* topK - integer - top "how many" to consider for the evaluation. By default top 10 items are to be considered

#### Output:
* DCG score

Don't forget, DCG is calculated for every user separately and then the average is returned.


<font color='red'>**Attention!**</font> Use logarithm with base 2 for discounts! Remember that the top1 recommendation shouldn't get discounted! Users without interactions in the test set shouldn't contribute to the score.

In [2]:
def get_dcg_score(predictions: np.ndarray, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    predictions - np.ndarray - predictions of the recommendation algorithm for each user.
    test_interaction_matrix - np.ndarray - test interaction matrix for each user.

    returns - float - mean dcg score over all user.
    """
    score = None

    # TODO: YOUR IMPLEMENTATION.

    score = 0

    # In case a interaction_row is completely empty
    delete_list = []
    for i, row in enumerate(test_interaction_matrix):
        if np.all((row == 0)):
            delete_list.append(i)

    test_interaction_matrix = np.delete(test_interaction_matrix, delete_list, 0)
    predictions = np.delete(predictions, delete_list, 0)

    for user in range(predictions.shape[0]):
        for sample in range(min(topK, predictions.shape[1])):
            if test_interaction_matrix[user][sample]:
                score += 1 / np.log2(2 + predictions[user][sample])

    return score / predictions.shape[0]

In [3]:
predictions = np.array([[0, 1, 2, 3], [3, 2, 1, 0]])
test_interaction_matrix = np.array([[1, 0, 0, 0], [0, 0, 0, 1]])

dcg_score = get_dcg_score(predictions, test_interaction_matrix, topK=4)

assert np.isclose(dcg_score, 1), "1 expected"

* Can DCG score be higher than 1?
* Can the average DCG score be higher than 1?
* Why?

### nDCG Score

Following the same parameter convention as for DCG implement nDCG metric.

<font color='red'>**Attention!**</font> Remember that ideal DCG is calculated separetely for each user and depends on the number of tracks held out for them as a Test set! Use logarithm with base 2 for discounts! Remember that the top1 recommendation shouldn't get discounted!

In [4]:
def get_ndcg_score(predictions: np.ndarray, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    predictions - np.ndarray - predictions of the recommendation algorithm for each user.
    test_interaction_matrix - np.ndarray - test interaction matrix for each user.
    topK - int - topK recommendations should be evaluated.
    
    returns - average ndcg score over all users.
    """
    score = None

    # TODO: YOUR IMPLEMENTATION.

    score = 0
    # In case a interaction_row is completely empty
    delete_list = []
    for i, row in enumerate(test_interaction_matrix):
        if np.all((row == 0)):
            delete_list.append(i)

    test_interaction_matrix = np.delete(test_interaction_matrix, delete_list, 0)
    predictions = np.delete(predictions, delete_list, 0)

    for user in range(predictions.shape[0]):
        maximum = np.sum(test_interaction_matrix[user])

        perfect_score = 0
        local_score = 0
        local_counter = 0  # To calculate the formula

        for sample in range(min(topK, predictions.shape[1])):

            if test_interaction_matrix[user, predictions[user, sample]]:
                local_score += 1 / np.log2(2 + local_counter)
            if sample < maximum:
                perfect_score += 1 / np.log2(2 + local_counter)
            local_counter += 1

        score += local_score / perfect_score

    return score / predictions.shape[0]

In [5]:
predictions = np.array([[0, 1, 2, 3], [3, 2, 1, 0], [1, 2, 3, 0], [-1, -1, -1, -1]])
test_interaction_matrix = np.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]])

ndcg_score = get_ndcg_score(predictions, test_interaction_matrix, topK=4)

assert np.isclose(ndcg_score, 1), "ndcg score is not correct."

* Can nDCG score be higher than 1?

### Average Artist Entropy

Calculate the metric of Diversity as the Average Artist entropy (see UE slides).
#### Parameters:
* predictions - as above for DCG and nDCG;
* item_df - dataframe containing 'artist' and 'track' columns, index - track id (use corresponding data file)
* topK - depth of the list to be evaluated, as before

#### Result:
Average Artist Entropy over users

Recap, main points:
* First calculate diversity for each user, then return the mean over users
* For every user build distribution of recommended tracks over artists (within topK). This distribution cannot have more than topK bins! Dont forget to turn it into probability distribution dividing it by topK
* Use the formula from the UE slides for the per-user entropy

<font color='red'>**Attention!**</font> Use logarithm with base 2!

In [6]:
def shannon_entropy(predictions, item_df, perfect=False):
    percentage = 1 / predictions.shape[0]
    entropy = 0

    if perfect:
        for sample in predictions:
            entropy -= percentage * np.log2(percentage)

    else:
        unique_actors = {item: 0 for item in np.unique(item_df.iloc[:, 0])}
        for sample in predictions:
            unique_actors[item_df.iloc[sample].values[0]] += 1

        total = predictions.shape[0]

        for value in unique_actors.values():
            if value == 0:
                continue
            entropy -= value / total * np.log2(value / total)

    return entropy


def get_average_entropy_score(predictions: np.ndarray, item_df: pd.DataFrame, topK=10) -> float:
    """
    predictions - np.ndarray - predictions of the recommendation algorithm for each user.
    item_df - pd.DataFrame - information about each song with columns 'artist' and 'track'.
    
    returns - float - average entropy score of the predictions.
    """

    score = None

    # TODO: YOUR IMPLEMENTATION.

    # In case a prediction_row is completely empty
    delete_list = []
    for i, row in enumerate(predictions):
        if np.all((row == -1)):
            delete_list.append(i)

    predictions = np.delete(predictions, delete_list, 0)

    if topK > predictions.shape[1]:
        raise IndexError
    score = 0

    for user in range(predictions.shape[0]):
        perfect_score = shannon_entropy(predictions[user, 0:min(topK, predictions.shape[1])], item_df, perfect=True)

        local_score = shannon_entropy(predictions[user, 0:min(topK, predictions.shape[1])], item_df)
        score += local_score / perfect_score
    return score / predictions.shape[0]

In [7]:
item_df = pd.DataFrame({'artist': ['A1', 'A1', 'A1', 'A1', 'A2', 'A3', 'A4']})
predictions = np.array([[0, 1, 2, 3], [6, 5, 4, 3], [-1, -1, -1, -1]])
avg_entr_score = get_average_entropy_score(predictions, item_df, topK=4)

assert np.isclose(avg_entr_score, 0.5), "average entropy score is not correct."

## <font color='red'>TASK 2/2</font>: Evaluation
Use provided rec.py (see imports below) to build a simple evaluation framework. It should be able to evaluate POP, ItemKNN (CF) and SVD.

In the end for each algorithm you should be able to obtain results formatted as follows:

```
{'m': {'ndcg': <>, 'average_entropy': <>},
 'f': {'ndcg': <>, 'average_entropy': <>},
 'all': {'ndcg': <>, 'average_entropy': <>}}
```
 
Every metric calculated for three groupls of test users: only for female, only for male and for all users together.
Every value should be an average, calculated over two different data splits.

In [8]:
from rec import svd_decompose, svd_recommend_to_list
from rec import inter_matr_binary, split_interactions
from rec import recTopK
from rec import recTopKPop

In [9]:
path = 'data_3/'

usr_path = path + 'sampled_1000_items_demo.txt'
itm_path = path + 'sampled_1000_items_tracks.txt'
inter_path = path + 'sampled_1000_items_inter.txt'

### (A) Universal Launcher
Receives a config as described, train and test interaction matrices.

Returns a matrix of size (total number of users) x (topK). cells - recommended item ids, sorted according to the score. Fill the cells corresponding to users with no interactions in the test set with (-1).

In [10]:
def get_recommendations_for_algorithm(config, inter_matrix_train, inter_matrix_test) -> np.ndarray:
    """
    inter_matrix_train - np.ndarray - test interaction matrix over all users and items.
    inter_matrix_test -  np.ndarray - train interaction matrix over all users and items.
    config - dict - configuration of this evaluation following the structure:

    config = {
        "algorithm": str - one of ['SVD', 'CF', 'TopPop']
        "inter_train_file_paths": str - inter train files over all splits,
        "inter_test_file_paths": str - inter test files over all splits,
        "user_file_path": str - usr_path,
        "item_file_path": str - itm_path,
        "top_k": int - number of recommendations to evaluate
        "n": int - used for CF.
        "f": int - length of hidden representations for SVD
    }

    returns - np.ndarray - list of recommendations for a specific algorithm in the shape (users, topK)
                           filled with -1 for users NOT in the test set and filled with topK
                           recommendations for users in the test set.
    """

    rec = None

    df_users = pd.read_csv(config['user_file_path'], sep='\t', header=None, names=['location', 'age', 'gender', 'date'])
    df_items = pd.read_csv(config['item_file_path'], sep='\t', header=None, names=['artist', 'track'])

    rec = np.full((len(df_users), config['top_k']), -1)

    # TODO: YOUR IMPLEMENTATION.
    algo = config['algorithm']
    users = list(range(inter_matrix_test.shape[0]))
    if algo == 'SVD':
        U, V = svd_decompose(inter_matrix_train, config["f"])

        user_seen_items = [[] for i in range(inter_matrix_test.shape[0])]
        for i, inter_row in enumerate(inter_matrix_train):
            for j, item in enumerate(inter_row):
                if item == 1:
                    user_seen_items[i].append(j)

        target_rec = svd_recommend_to_list(users, user_seen_items, U, V, config['top_k'])
        for i, user in enumerate(users):
            rec[user] = target_rec[i]

    elif algo == 'CF':

        for user in users:
            rec[user], _ = recTopK(inter_matrix_train, user, config['top_k'], config['n'])


    elif algo == 'TopPop':
        for user in users:
            rec[user] = recTopKPop(inter_matrix_train, user, config['top_k'])

    else:
        raise IndexError('wrong Algorithm Key')
    return rec

### Single set evaluation (already implemented)

In [11]:
def evaluate_predictions(predictions: np.ndarray, test_interaction_matrix: np.ndarray,
                         item_df: pd.DataFrame, topK=10) -> dict:
    """
    This function returns a dictinary with all scores of predictions.

    predictions - np.ndarray - predictions of the algorithm over all users.
    test_interaction_matrix - np.ndarray - test interaction matrix over all users and items.
    item_df - pd.DataFrame - information about each item with columns: 'artist', 'track'
    topK - int - topK prediction should be evaluated

    returns - dict - calculated metric scores, contains keys "ndcg" and "average_entropy".
    """

    metrics = {}
    ndcg = get_ndcg_score(predictions, test_interaction_matrix, topK)
    metrics['ndcg'] = ndcg

    average_entropy = get_average_entropy_score(predictions, item_df, topK)
    metrics['average_entropy'] = average_entropy

    return metrics

### (B) User-group evaluation

In [12]:
def evaluate_gender(predictions: np.ndarray, test_interaction_matrix: np.ndarray, user_df: pd.DataFrame,
                    item_df: pd.DataFrame, num_users=500, topK=10) -> dict:
    """
    This function will evaluate certain predictions for each gender individually and return a dictionary
    following the structure:

    {'gender_key': {'metric_key': metric_score}}

    predictions - np.ndarray - predictions of the algorithm over all users.
    test_interaction_matrix - np.ndarray - test interaction matrix over all users and items.
    user_df - pd.DataFrame - information about each user with columns: location', 'age', 'gender', 'date'
    item_df - pd.DataFrame - information about each item with columns: 'artist', 'track'
    topK - int - topK prediction should be evaluated

    returns - dict - calculated metric scores for each gender.
    """

    metrics = {}

    # TODO: YOUR IMPLEMENTATION.
    female_users = []
    male_users = []

    for index in range(user_df.shape[0]):
        if user_df['gender'][index] == 'm':
            male_users.append(index)
        elif user_df['gender'][index] == 'f':
            female_users.append(index)
    female_prediction = predictions[female_users]
    male_prediction = predictions[male_users]

    ## adjust test_interaction_matrix to only predicted users;

    metrics['m'] = evaluate_predictions(male_prediction, test_interaction_matrix[male_users], item_df, topK)

    metrics['f'] = evaluate_predictions(female_prediction, test_interaction_matrix[female_users], item_df, topK)

    metrics['all'] = evaluate_predictions(predictions, test_interaction_matrix, item_df, topK)

    return metrics

### (C) Main evaluation function
Interprets the config and returns evaluation report for a single algorithm:
```
{'m': {'ndcg': <>, 'average_entropy': <>},
 'f': {'ndcg': <>, 'average_entropy': <>},
 'all': {'ndcg': <>, 'average_entropy': <>}}
```

Please pay attention to how splits are created and saved into the corresponding variables (Split Data section below)

In [13]:
def evaluate_algorithm(config) -> dict:
    """
    This function will evaluate a certain algorithm defined with the parameters in config by:
    - going over all test and train files
    - generating the recommendations for each data split
    - calling evaluate gender to get the metrics for each recommendation for each data split

    Then the average score for each gender and metric should be calculated over all data splits and
    a dictionary should be returned following the structure:
    {'gender_key': {'metric_key': avg_metric_score}}

    config - dict - configuration of this evaluation following the structure:

    config = {
        "algorithm": str - one of ['SVD', 'CF', 'TopPop']
        "inter_train_file_paths": str - array of inter train file paths (1 per split),
        "inter_test_file_paths": str - array of inter test file paths (1 per split),
        "user_file_path": str - usr_path,
        "item_file_path": str - itm_path,
        "top_k": int - number of recommendations to evaluate
        "n": int - used for CF.
        "f": int - length of hidden representations for SVD
    }

    returns - dict - average score of each metric for each gender over all data splits.
    """

    metrics = {}
    # TODO: YOUR IMPLEMENTATION.
    filepaths_train = config['inter_train_file_paths']
    filepaths_test = config['inter_test_file_paths']

    songs = pd.read_csv(config['item_file_path'], delimiter='\t', header=None, names=['Artist', 'Title'])
    users = pd.read_csv(config['user_file_path'], delimiter='\t', header=None,
                        names=['location', 'age', 'gender', 'date'])

    my_dict = {'m': {'ndcg': 0, 'average_entropy': 0}, 'f': {'ndcg': 0, 'average_entropy': 0},
               'all': {'ndcg': 0, 'average_entropy': 0}}

    for i in range(len(filepaths_test)):
        my_train_matrix = inter_matr_binary(config['user_file_path'], config['item_file_path'], filepaths_train[i])
        my_test_matrix = inter_matr_binary(config['user_file_path'], config['item_file_path'], filepaths_test[i])

        rec = get_recommendations_for_algorithm(config, my_train_matrix, my_test_matrix)
        my_dict2 = evaluate_gender(rec, my_test_matrix, users, songs)
        for ele in my_dict:
            for ele2 in my_dict[ele]:
                my_dict[ele][ele2] += my_dict2[ele][ele2] / len(filepaths_test)

    metrics = my_dict
    return metrics

### Splitting Data (already implemented)

In [15]:
train_inter_files = []
test_inter_files = []

num_splits = 2
p_i = 0.3
p_u = 0.5

user_file_path = None
inter_file_path = None

user_file_path = usr_path
inter_file_path = inter_path

for i in range(num_splits):
    split_interactions(inter_file=inter_file_path,
                       user_file_path=user_file_path,
                       p_u=p_u,
                       p_i=p_i,
                       res_test_file="inter_TEST_" + str(i) + ".txt",
                       res_train_file="inter_TRAIN_" + str(i) + ".txt")

    train_inter_files.append("inter_TRAIN_" + str(i) + ".txt")
    test_inter_files.append("inter_TEST_" + str(i) + ".txt")

Train data_2:  6982
Test data_2:  1213
Train data_2:  6968
Test data_2:  1227


In [17]:
assert len(train_inter_files) == num_splits, "Number of Train files do not match the requirement"
assert len(test_inter_files) == num_splits, "Number of Test files do not match the requirement"

### Evaluating Every Algorithm
Make sure everything works. Try running evaluation with the three configs below.
We expect KNN to outperform other algorithms on our small data sample.

In [18]:
# Evaluate TopPop
config = {
    "algorithm": "TopPop",  # ['SVD', 'CF', 'TopPop']
    "inter_train_file_paths": train_inter_files,
    "inter_test_file_paths": test_inter_files,
    "user_file_path": usr_path,
    "item_file_path": itm_path,
    "top_k": 10,  # number of recommendations.
    "n": 5,  # used for CF.
    "f": 32,  # length of hidden representations
}

scores = evaluate_algorithm(config)
scores

{'m': {'ndcg': 0.09798054431234463, 'average_entropy': 1.0},
 'f': {'ndcg': 0.10735625527582683, 'average_entropy': 1.0},
 'all': {'ndcg': 0.0988300214801769, 'average_entropy': 1.0}}

In [19]:
# Evaluate Item KNN (CF)
config = {
    "algorithm": "CF",  # ['SVD', 'CF', 'TopPop']
    "inter_train_file_paths": train_inter_files,
    "inter_test_file_paths": test_inter_files,
    "user_file_path": usr_path,
    "item_file_path": itm_path,
    "top_k": 10,  # number of recommendations.
    "n": 5,  # used for CF.
    "f": 32,  # length of hidden representations
}

scores = evaluate_algorithm(config)
scores

{'m': {'ndcg': 0.21700279518014298, 'average_entropy': 0.9993341271985312},
 'f': {'ndcg': 0.1594155151195772, 'average_entropy': 1.0},
 'all': {'ndcg': 0.21178515571156825, 'average_entropy': 0.9993949145815799}}

In [20]:
# Evaluate SVD
config = {
    "algorithm": "SVD",  # ['SVD', 'CF', 'TopPop']
    "inter_train_file_paths": train_inter_files,
    "inter_test_file_paths": test_inter_files,
    "user_file_path": usr_path,
    "item_file_path": itm_path,
    "top_k": 10,  # number of recommendations.
    "n": 5,  # used for CF.
    "f": 256,  # length of hidden representations
}

scores = evaluate_algorithm(config)
scores

{'m': {'ndcg': 0.03523934615472972, 'average_entropy': 0.9998057870995716},
 'f': {'ndcg': 0.024935414779959357, 'average_entropy': 1.0},
 'all': {'ndcg': 0.03430576847983441, 'average_entropy': 0.9998235167529608}}

In [21]:
g_keys = ['m', 'f', 'all']
m_keys = ['ndcg', 'average_entropy']

assert all([k in g_keys for k in scores.keys()]), 'keys error'
assert all([k in m_keys for k in scores['m'].keys()]), 'keys error'
assert scores['all']['ndcg'] >= 0, "metric score should be a number."
assert scores['all']['average_entropy'] >= 0, "metric score should be a number."

## Questions and Potential Future Work
* Do all algorithms treat Female and Male users similarly? Why?
* How would you try improve performance of all three algorithms?
* What other metrics would you consider to compare these recommender systems?
* What other user groups would you investigate?

In [20]:
# The end.